# 時系列データとは何か，その基本的な取り扱いについて調べる(2022/01/25)
---
## 概要
---
データの形式の一つである時系列データについて，その特性・分析手法(トレンド，周期性，ノイズ，移動平均，季節調整，階差，スペクトル，自己相関，相互相関，自己回帰モデル)について調べる．

## 時系列データ
---
時系列データとは，時間の経過とともに変化する量を等間隔で記録したデータのことである．例えば，毎日測定する体重の推移であるとか，株価，商品の売り上げ，気温，そして今回例として使用するGDPの推移に至るまで，時系列データは様々な事象から得られるものである．

時系列データはその性質ゆえ，グラフにプロットされる際は横軸が時間軸，そして縦軸が当該データの特徴量になることが多い．時間軸のスケールは扱うデータの性質によってさまざまで，一秒おきに得られるデータもあれば，数年おきに得られるデータもある．

このnotebookでは四半期ごとに収集されていた日本国の実質GDPを例として，時系列データに対する分析の切り口を学ぶ．

早速，内閣府が公表している実質GDPのデータをpandas DataFrameに読み込んでいく．

In [1]:
import pandas as pd

gdp_jp = pd.read_excel('gdp_jp_2021.xlsx',
                       header=None,
                       skiprows=7,
                       names=['y/m', 'GDP'],
                       index_col=[0],
                       usecols=[0, 1])
gdp_jp.dropna(inplace=True)
gdp_jp

,GDP
y/m,
1994/ 1- 3.,109110.2
4- 6.,107756.4
7- 9.,112077.0
10-12.,117578.7
1995/ 1- 3.,110520.9
...,...
7- 9.,130613.2
10-12.,138078.0
2021/ 1- 3.,135013.1


### トレンド
### 周期性
### ノイズ

## 時系列データの分析
---
### 移動平均
### 季節調整
### 階差
### スペクトル
### 自己相関
### 相互相関
### 自己回帰モデル

## 参考文献

[時系列，Wikipedia](https://ja.wikipedia.org/wiki/%E6%99%82%E7%B3%BB%E5%88%97)

[時系列データの解析，東京大学　数理・情報教育研究センター](http://www.mi.u-tokyo.ac.jp/consortium2/pdf/4-4_literacy_level_note.pdf)

[32-3. 時系列データと移動平均，統計WEB](https://bellcurve.jp/statistics/course/12933.html)

[32-4. 時系列データにおける周期変動，統計WEB](https://bellcurve.jp/statistics/course/23739.html)

[32-5. 自己相関，統計WEB](https://bellcurve.jp/statistics/course/12935.html)

[【テックコラム】時系列データの基礎と可視化，DataCurrent](https://www.datacurrent.co.jp/column/time-series-visualization20210628/)

[時系列相関，株式会社アイスタット](https://istat.co.jp/ji_commentary/time_series)

[統計表一覧(2021年 7-9月期 2次速報値)，内閣府](https://www.esri.cao.go.jp/jp/sna/data/data_list/sokuhou/files/2021/qe213_2/gdemenuja.html)

[pandas.read_excel，pandas 1.4.0 documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html)

[pandas.DataFrame.dropna，pandas 1.4.0 documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)